## Understanding the differences between concat, join, and merge

- `merge` and `join` DataFrame(and not Series) methods and the concat method


- concat:
    * Pandas funtion
    * Combines two or more pandas objects vertically or horizontally
    * Align only on the index
    * Errors whenever a duplicate appears in the index
    * By default outer join with option for inner
    
    
- join:
    * DataFrame method
    * Combines two or more pandas objects horizontally
    * Algins the calling DataFrame's column(s) or index with the other objects' index
    * Handles duplicate values on the joining columns/index by performing a cartesian product
    * By default left join
    
    
- merge:
    * DataFrame method
    * Combines exactly two DataFrames horizontally
    * Aligns the calling DataFrame's column(s)/index with the other DataFrame's column(s)/index
    * Handles duplicate values on the joining columns/index by performing a cartesian product
    * By default inner join


In [1]:
import pandas as pd

## Aligning the values of index or column labels

In [7]:
years = 2016, 2017, 2018
stock_tables = [pd.read_csv('data/stocks_{}.csv'.format(year), index_col='Symbol') for year in years]
for stock in stock_tables:
    print(stock)

stocks_2016, stocks_2017, stocks_2018 = stock_tables

        Shares  Low  High
Symbol                   
AAPL        80   95   110
TSLA        50   80   130
WMT         40   55    70
        Shares  Low  High
Symbol                   
AAPL        50  120   140
GE         100   30    40
IBM         87   75    95
SLB         20   55    85
TXN        500   15    23
TSLA       100  100   300
        Shares  Low  High
Symbol                   
AAPL        40  135   170
AMZN         8  900  1125
TSLA        50  220   400


### concat

concat method is the only one able to combine DataFrames vertically

In [4]:
pd.concat(stock_tables, keys=[2016, 2017, 2018])

Shares  Low  High
     Symbol                   
2016 AAPL        80   95   110
     TSLA        50   80   130
     WMT         40   55    70
2017 AAPL        50  120   140
     GE         100   30    40
     IBM         87   75    95
     SLB         20   55    85
     TXN        500   15    23
     TSLA       100  100   300
2018 AAPL        40  135   170
     AMZN         8  900  1125
     TSLA        50  220   400

It can also combine DataFrames horizontally by changing the axis parameter to columns

In [5]:
pd.concat(dict(zip(years, stock_tables)), axis='columns')

2016                2017                 2018               
     Shares   Low   High Shares    Low   High Shares    Low    High
AAPL   80.0  95.0  110.0   50.0  120.0  140.0   40.0  135.0   170.0
AMZN    NaN   NaN    NaN    NaN    NaN    NaN    8.0  900.0  1125.0
GE      NaN   NaN    NaN  100.0   30.0   40.0    NaN    NaN     NaN
IBM     NaN   NaN    NaN   87.0   75.0   95.0    NaN    NaN     NaN
SLB     NaN   NaN    NaN   20.0   55.0   85.0    NaN    NaN     NaN
TSLA   50.0  80.0  130.0  100.0  100.0  300.0   50.0  220.0   400.0
TXN     NaN   NaN    NaN  500.0   15.0   23.0    NaN    NaN     NaN
WMT    40.0  55.0   70.0    NaN    NaN    NaN    NaN    NaN     NaN

### join

- Use the join method to combine the stock_2016 and stock_2017 DataFrames
- If any of the columns have the same names, then you must supply a value to the lsuffix of rsuffix parameters to distinguish them in the result

In [8]:
stocks_2016.join(stocks_2017, lsuffix='_2016', rsuffix='_2017', how='outer')

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017
Symbol,,,,,,
AAPL,80.0,95.0,110.0,50.0,120.0,140.0
GE,NaN,NaN,NaN,100.0,30.0,40.0
IBM,NaN,NaN,NaN,87.0,75.0,95.0
SLB,NaN,NaN,NaN,20.0,55.0,85.0
TSLA,50.0,80.0,130.0,100.0,100.0,300.0
TXN,NaN,NaN,NaN,500.0,15.0,23.0
WMT,40.0,55.0,70.0,NaN,NaN,NaN


Replicate the output of the concat method exactly

In [9]:
other = [stocks_2017.add_suffix('_2017'),
         stocks_2018.add_suffix('_2018')]
stocks_2016.add_suffix('_2016').join(other, how='outer')

,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017,Shares_2018,Low_2018,High_2018
AAPL,80.0,95.0,110.0,50.0,120.0,140.0,40.0,135.0,170.0
AMZN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,900.0,1125.0
GE,NaN,NaN,NaN,100.0,30.0,40.0,NaN,NaN,NaN
IBM,NaN,NaN,NaN,87.0,75.0,95.0,NaN,NaN,NaN
SLB,NaN,NaN,NaN,20.0,55.0,85.0,NaN,NaN,NaN
TSLA,50.0,80.0,130.0,100.0,100.0,300.0,50.0,220.0,400.0
TXN,NaN,NaN,NaN,500.0,15.0,23.0,NaN,NaN,NaN
WMT,40.0,55.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN


Check whether they actually are exactly equal

In [12]:
stock_join = stocks_2016.add_suffix('_2016').join(other, how='outer')

stock_concat = pd.concat(dict(zip(years, stock_tables)), axis=1)

level_1 = stock_concat.columns.get_level_values(1)
level_0 = stock_concat.columns.get_level_values(0).astype(str)
stock_concat.columns = level_1 + '_' + level_0
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.equals.html
stock_join.equals(stock_concat)

True

### merge

- By default `merge` attempts to align the values in the columns that have the same name for each of the DataFrames
- You can choose to have it align on the index by setting the boolean parameters `left_index` and `right_index` to True

In [13]:
stocks_2016.merge(stocks_2017, left_index=True, right_index=True)

,Shares_x,Low_x,High_x,Shares_y,Low_y,High_y
Symbol,,,,,,
AAPL,80,95,110,50,120,140
TSLA,50,80,130,100,100,300


By default, `merge` uses an inner join and automatically supplies suffixes for identically named columns

In [16]:
step1 = stocks_2016.merge(stocks_2017, left_index=True, right_index=True, how='outer', suffixes=('_2016', '_2017'))

stock_merge = step1.merge(stocks_2018.add_suffix('_2018'), left_index=True, right_index=True, how='outer')
stock_concat.equals(stock_merge)

True

## Aligning the values of columns

In [18]:
names = ['prices', 'transactions']
food_tables = [pd.read_csv('data/food_{}.csv'.format(name)) for name in names]
food_prices, food_transactions = food_tables
food_prices

,item,store,price,Date
0,pear,A,0.99,2017
1,pear,B,1.99,2017
2,peach,A,2.99,2017
3,peach,B,3.49,2017
4,banana,A,0.39,2017
5,banana,B,0.49,2017
6,steak,A,5.99,2017
7,steak,B,6.99,2017
8,steak,B,4.99,2015


In [19]:
food_transactions

,custid,item,store,quantity
0,1,pear,A,5
1,1,banana,A,10
2,2,steak,B,3
3,2,pear,B,1
4,2,peach,B,2
5,2,steak,B,1
6,2,coconut,B,4


Find the total amount of each transaction

In [20]:
food_transactions.merge(food_prices, on=['item', 'store'])

,custid,item,store,quantity,price,Date
0,1,pear,A,5,0.99,2017
1,1,banana,A,10,0.39,2017
2,2,steak,B,3,6.99,2017
3,2,steak,B,3,4.99,2015
4,2,steak,B,1,6.99,2017
5,2,steak,B,1,4.99,2015
6,2,pear,B,1,1.99,2017
7,2,peach,B,2,3.49,2017


- Customer 2 has a total of four `steak` items. As the `steak` item appears twice in each table for store B.
- A Cartesian product takes place between them, resulting in four rows.
- Also, `coconut` item is missing because theres was no corresponding price for it

In [26]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html
# https://kongdols-room.tistory.com/120
# row를 조건에 맞게 필터링, []을 이용해도 되지만 대규모 데이터에서는 query 메소드가 성능 면에서 우위

# 최신 가격만 합치기 위해서 query 메소드로 2017년 가격으로 필터링
# merge는 기본적으로 inner join, coconut은 transactions에만 있기 때문에 inner join시 누락
food_transactions.merge(food_prices.query('Date == 2017'), how='left')

,custid,item,store,quantity,price,Date
0,1,pear,A,5,0.99,2017.0
1,1,banana,A,10,0.39,2017.0
2,2,steak,B,3,6.99,2017.0
3,2,pear,B,1,1.99,2017.0
4,2,peach,B,2,3.49,2017.0
5,2,steak,B,1,6.99,2017.0
6,2,coconut,B,4,NaN,NaN


In [22]:
food_prices_join = food_prices.query('Date == 2017').set_index(['item', 'store'])
food_prices_join

price  Date
item   store             
pear   A       0.99  2017
       B       1.99  2017
peach  A       2.99  2017
       B       3.49  2017
banana A       0.39  2017
       B       0.49  2017
steak  A       5.99  2017
       B       6.99  2017

- join 메소드는 passed DataFrame의 index에 대해서만 align
- join 메소드를 호출하는 DataFrame(calling DataFrame)의 index나 columns를 사용할 수 있음
- `on` parameter 필요

In [24]:
food_transactions.join(food_prices_join, on=['item', 'store'])

,custid,item,store,quantity,price,Date
0,1,pear,A,5,0.99,2017.0
1,1,banana,A,10,0.39,2017.0
2,2,steak,B,3,6.99,2017.0
3,2,pear,B,1,1.99,2017.0
4,2,peach,B,2,3.49,2017.0
5,2,steak,B,1,6.99,2017.0
6,2,coconut,B,4,NaN,NaN


To replicate this with the `concat` method, you would need to put the item and store columns into the index of both DataFrames

In [27]:
pd.concat([food_transactions.set_index(['item', 'store']), food_prices.set_index(['item', 'store'])], axis='columns')

Exception: cannot handle a non-unique multi-index!